## Predict features' categories and evaluate performance (bluehouse)

In [1]:
import os
import openai
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay
import sklearn
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,classification_report
import numpy as np
import matplotlib.pyplot as plt
import random
openai.api_key = os.getenv("OPENAI_API_KEY")

## Get predictions

In [2]:
df=pd.read_csv("../../input_data/complete_file.csv")


preprompt="Consider this situation and answer the question that follows:\nSituation: "

df=df.assign(statement=preprompt+df.scenario+"\nQuestion: ")

df=df.loc[(df.study=='blue.house') ,:].reset_index(drop=True)

In [3]:
compr="How much would it cost to reverse this damage? (Respond with a dollar amount.)"
f3="How much money will Hank get from the stranger?"


In [4]:
for i,d in df.iterrows():
    input_prompt=d['statement']+compr
    completion = openai.Completion.create(engine="text-davinci-002", prompt=input_prompt,temperature=0,max_tokens=60)
    
    df.loc[i,'first_response']=completion.choices[0].text
    input_2=input_prompt+completion.choices[0].text+"\nQuestion: "+f3+"\n"
    second_response = openai.Completion.create(engine="text-davinci-002", prompt=input_2,temperature=0,max_tokens=60)
    df.loc[i,'second_response']=second_response.choices[0].text


In [5]:
blue_house=df.copy()

In [6]:
blue_house['gpt_feature2']=blue_house.first_response
blue_house.gpt_feature2=blue_house.gpt_feature2.replace("It would cost around one hundred and fifty dollars","150",regex=True)
blue_house.gpt_feature2=blue_house.gpt_feature2.replace("It would cost around one hundred dollars","100",regex=True)
blue_house['gpt_feature2']=blue_house.gpt_feature2.str.extract('(\d+[,\d]*)')
blue_house['gpt_feature2']=blue_house.gpt_feature2.replace(",","",regex=True)

In [7]:
blue_house.gpt_feature2.value_counts()

100        27
1000        8
100000      7
10000       4
1000000     4
0           3
80          1
Name: gpt_feature2, dtype: int64

In [8]:
blue_house['gpt_feature2']=pd.to_numeric(blue_house['gpt_feature2'])

blue_house['gpt_feature3']=blue_house.second_response
blue_house.gpt_feature3=blue_house.gpt_feature3.replace("Hank will get one hundred dollars from the stranger","100",regex=True)
blue_house.gpt_feature3=blue_house.gpt_feature3.replace("Hank will get one hundred thousand dollars from the stranger","100000",regex=True)
blue_house.gpt_feature3=blue_house.gpt_feature3.replace("Hank will get ten thousand dollars from the stranger","10000",regex=True)
blue_house.gpt_feature3=blue_house.gpt_feature3.replace("Hank will get one thousand dollars from the stranger","1000",regex=True)
blue_house['gpt_feature3']=blue_house.gpt_feature3.str.extract('(\d+[,\d]*)')
blue_house['gpt_feature3']=blue_house.gpt_feature3.replace(",","",regex=True)

blue_house['gpt_feature3']=pd.to_numeric(blue_house['gpt_feature3'])

In [9]:
blue_house.gpt_feature3.value_counts()

1000.0       14
100.0        12
100000.0      9
10000.0       9
1000000.0     7
Name: gpt_feature3, dtype: int64

In [10]:
blue_house.to_csv("../../outputs/analysis/features/bluehouse_features.csv")

In [84]:
#blue=blue_house.loc[:,['context', 'condition', 'study', 'human.response', 'feature1',
#       'feature2', 'gpt_feature2','feature3', 'gpt_feature3','feature1b', 'feature1c', 'prompt',
#       'prompt_short', 'prompt_long','first_response']]

#blue.to_csv("./bluehouse_features_categorical1.csv",index=False)